# Data Extraction from MODS folder

*note* 
- Data extraction is performed in BMI cluster which runs in
    - python: 3.6.12.final.0
    - pandas: 1.1.4
- The installed libraries ver. in BMI cluster is different from what VM has. Thus, some parts of the scripts are modified to accommodate this. 


The following notebook outputs data creates a supertable in hourly bins

1. Encounters
2. Sepsis Label Data
    - merge sepsis label data
    - filter 2021 data and define cohort csns
3. Vitals
4. Labs
5. GCS
6. Demographics
7. Fluids
8. Mechanical Ventilation

In [8]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import os
from tqdm import tqdm
import sys 

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
pd.options.display.float_format = '{:,.4f}'.format

## Extraction Helper Functions

In [4]:
def import_dsv(directories, fetch_cols = [], thresholds = [], csns = [], pat_ids = [], save_dir = ""):
    
    total = pd.DataFrame()
    processed_cols = []
    if len(thresholds)!=0:
        process_cols = list(thresholds.keys())
    else: process_cols = []
        
    
    # read in for each directory
    for directory in directories:
        
        chunksize = 10**5

        t = int(os.path.getsize(directory)/chunksize) + 1
        
        with tqdm(total = t, file = sys.stdout) as pbar:
            
            # read in by chunk size
            for i, chunk in enumerate(pd.read_csv(directory, sep="|", chunksize=chunksize, low_memory=False)):
                
                # if pat_ids and csns are specifed, just extract those patients. Otherwise, extract all patients.
                if len(pat_ids)!=0:
                    chunk = chunk[chunk.pat_id.isin(pat_ids)]
                if len(csns)!= 0:
                    chunk = chunk[chunk.csn.isin(csns)]
                
                # extract only specified columns. Otherwise, extract all columns.
                if len(fetch_cols) != 0:
                    chunk = chunk[fetch_cols]
               
                # if there's anything to preprocess- data cleaning w/ thresholds
                for feature in process_cols:
                    chunk.loc[:,feature] = chunk[feature].replace(r'\>|\<|\%|\/|\s','',regex=True)
                    chunk.loc[:,feature] = pd.to_numeric(chunk[feature], errors='coerce')
                    mask_ind = (chunk[feature] < thresholds[feature][1]) & (chunk[feature] > thresholds[feature][0])
                    chunk.loc[~mask_ind, feature]  = np.nan

                total = total.append(chunk)

                pbar.set_description('Importing: %d' % (1 + i))
                pbar.update(1)
    
    print("extraction complete")
    
    if len(save_dir) != 0:
        total.to_csv(save_dir, index = False)
    
    return total.reset_index(drop = True)

In [5]:
def import_labs(directories, thresholds = [], csns = [], pat_ids = [], loinc_dir = "", save_dir = ""):
    
    fetch_cols = ["pat_id", "csn", "collection_time", "component", "component_id", "loinc_code", "lab_result"]
    total = pd.DataFrame()
    loinc_codes = 0
    any_codes = []
    specific_codes = []
    if len(thresholds)!=0:
        process_cols = list(thresholds.keys())
    else: process_cols = []
        
    if len(loinc_dir) > 0 :  
        loinc_codes = pd.read_csv(loinc_dir)
        any_codes = list(loinc_codes[loinc_codes.isna().any(axis = 1)]["loinc_code"])
        codes = list(loinc_codes["component_id"])
        specific_codes = list(loinc_codes[~loinc_codes.isna().any(axis = 1)]["loinc_code"])
    
    # read in for each directory
    for directory in directories:
        
        chunksize = 10**5

        t = int(os.path.getsize(directory)/chunksize) + 1
        
        with tqdm(total = t, file = sys.stdout) as pbar:
            
            # read in by chunk size
            for i, chunk in enumerate(pd.read_csv(directory, sep="|", chunksize=chunksize, low_memory=False)):
                
                # if pat_ids and csns are specifed, just extract those patients. Otherwise, extract all patients.
                if len(pat_ids)!=0:
                    chunk = chunk[chunk.pat_id.isin(pat_ids)]
                if len(csns)!= 0:
                    chunk = chunk[chunk.csn.isin(csns)]
  
                # for labs only
                if len(any_codes) + len(specific_codes) != 0:
                    chunk1 = chunk[chunk["component_id"].isin(codes)]
                    chunk1 = chunk1[chunk1["loinc_code"].isin(specific_codes)]
                    chunk2 = chunk[chunk["loinc_code"].isin(any_codes)]
                    chunk = pd.concat([chunk1, chunk2])

                total = total.append(chunk)
                
                pbar.set_description('Importing: %d' % (1 + i))
                pbar.update(1)  
    
    if len(loinc_dir) > 0:    
        labs_cleaned = total.merge(loinc_codes, on = "loinc_code", how = "left")
    else:
        labs_cleaned["FEATURE"] = total["component"]
        
    labs_cleaned = labs_cleaned[["pat_id", "csn", "collection_time", "FEATURE", "lab_result"]]
    labs_cleaned["lab_result"] = labs_cleaned["lab_result"].replace(r'\>|\<|\%|\/|\s','',regex=True)
    labs_cleaned["lab_result"] = pd.to_numeric(labs_cleaned["lab_result"], errors='coerce')
    piv_new_labs = labs_cleaned.pivot_table(index=['csn', "pat_id", "collection_time"], columns='FEATURE', values='lab_result')
    piv_new_labs = piv_new_labs.reset_index()


    for feature in process_cols:
        mask_ind = (piv_new_labs[feature] < thresholds[feature][1]) & (piv_new_labs[feature] > thresholds[feature][0])
        piv_new_labs.loc[~mask_ind, feature]  = np.nan
                
    
    print("extraction complete")
    
    if len(save_dir) > 0:
        piv_new_labs.to_csv(save_dir, index = False)
    
    return piv_new_labs


In [31]:
def import_csv(directories, fetch_cols = [], thresholds = [], csns = [], pat_ids = [], save_dir = ""):
    
    total = pd.DataFrame()
    processed_cols = []
    if len(thresholds)!=0:
        process_cols = list(thresholds.keys())
    else: process_cols = []
        
    
    # read in for each directory
    for directory in directories:
        
        chunksize = 10**5

        t = int(os.path.getsize(directory)/chunksize) + 1
        
        with tqdm(total = t, file = sys.stdout) as pbar:
            
            # read in by chunk size
            for i, chunk in enumerate(pd.read_csv(directory, chunksize=chunksize, low_memory=False)):
                
                # if pat_ids and csns are specifed, just extract those patients. Otherwise, extract all patients.
                if len(pat_ids)!=0:
                    chunk = chunk[chunk.pat_id.isin(pat_ids)]
                if len(csns)!= 0:
                    chunk = chunk[chunk.CSN.isin(csns)]
                
                # extract only specified columns. Otherwise, extract all columns.
                if len(fetch_cols) != 0:
                    chunk = chunk[fetch_cols]
               
                # if there's anything to preprocess- data cleaning w/ thresholds
                for feature in process_cols:
                    chunk.loc[:,feature] = chunk[feature].replace(r'\>|\<|\%|\/|\s','',regex=True)
                    chunk.loc[:,feature] = pd.to_numeric(chunk[feature], errors='coerce')
                    mask_ind = (chunk[feature] < thresholds[feature][1]) & (chunk[feature] > thresholds[feature][0])
                    chunk.loc[~mask_ind, feature]  = np.nan

                total = total.append(chunk)

                pbar.set_description('Importing: %d' % (1 + i))
                pbar.update(1)
    
    print("extraction complete")
    
    if len(save_dir) != 0:
        total.to_csv(save_dir, index = False)
    
    return total.reset_index(drop = True)

## 1. Read Encounters
Include patients from EUH Midtown, Emory University Hospital, Emory Saint Josephs Hospital, and Emory Johns Creek Hospital

In [9]:
f = ["/labs/kamaleswaranlab/MODS/Data/Emory_Data/em_data/2020/CJSEPSIS_ENCOUNTER_2020.dsv",
     "/labs/kamaleswaranlab/MODS/Data/Emory_Data/em_data/2021/CJSEPSIS_ENCOUNTER_2021.dsv"]
fetch_cols = ["pat_id", "csn", "age", "ed_presentation_time", "hospital_admission_date_time", 
              "hospital_discharge_date_time", "facility_nm", "discharge_to"]
total_enc = import_dsv(directories = f, fetch_cols = fetch_cols)

Importing: 3:   1%|          | 3/425 [00:03<08:12,  1.17s/it]
extraction complete


In [10]:
total_enc.facility_nm.value_counts()

EUH Midtown                     150655
Emory University Hospital       124261
Emory Saint Josephs Hospital     89666
Emory Johns Creek Hospital       59531
Name: facility_nm, dtype: int64

In [11]:
# Preprocess encounter data

# Change facility name to abbreviated names
facility_map = {"Emory University Hospital": "EUH", 
                'Emory Johns Creek Hospital': "EJCH",
                'Emory Saint Josephs Hospital': "ESJH", 
                'EUH Midtown': "EUHM"}
total_enc = total_enc.replace({"facility_nm": facility_map})

# Change datatype
total_enc["hospital_admission_date_time"] = pd.to_datetime(total_enc["hospital_admission_date_time"])
# Create column for year
total_enc["year"] = total_enc["hospital_admission_date_time"].dt.year


In [12]:
# filter only legacy hospitals
legacy = ["EUH", "EJCH", "EUHM", "ESJH"]
total_enc = total_enc[total_enc["facility_nm"].isin(legacy)]

## 2. Read Sepsis Label Data

In [13]:
ehr = pd.read_csv("../real_time_sepsis_development/real_time_data/ehr_data.csv")
ehr = ehr.rename(columns ={"Encounter": "csn"})

ehr["hospital_admission_date"] = pd.to_datetime(ehr["hospital_admission_date"])
ehr["time_zero"] = pd.to_datetime(ehr["time_zero"])

ehr = ehr[~ehr.time_zero.isna()]
ehr = ehr.sort_values(by = "time_zero")
ehr = ehr.drop_duplicates(subset = ["csn"], keep = "first")

#csns = ehr.csn.unique()
print("there are %i csns from EHR data" % len(ehr))


there are 6298 csns from EHR data


### 2.1 Merge Sepsis Label Data to Encounters

In [16]:
merged_times = total_enc.merge(ehr, how = "left", on = ["csn", "facility_nm"])
merged_times = merged_times.drop(["hospital_admission_date", "hospital_discharge_date", "facility_nm"], axis = 1)
merged_times = merged_times.drop_duplicates(subset = ["csn"])

times = merged_times.copy()
times = times.rename(columns = {"time_zero": "t_sepsis3"})
times = times.sort_values("hospital_admission_date_time")

# we only want first encounter per patient
keep_csns = list(times.groupby("pat_id").csn.first())
times = times[times.csn.isin(keep_csns)]
print("total number of csns:" , len(keep_csns))

total number of csns: 252003


### 2.2 Filter 2021 data only & define all csns to keep

In [23]:

times = times[times.year == 2021]
sepsis_2021 = list(times[~times.t_sepsis3.isna()].csn)
nosepsis_2021 = list(times[times.t_sepsis3.isna()].csn)
print("number of sepsis patients from 2021: ", len(sepsis_2021))
print("number of non-sepsis patients from 2021: ",len(nosepsis_2021))

csns = sepsis_2021 + nosepsis_2021
pat_ids = list(times.pat_id)

print("total number of encounters in 2021 cohort: ", len(csns))

number of sepsis patients from 2021:  1736
number of non-sepsis patients from 2021:  111365
total number of encounters in 2021 cohort:  113101


In [19]:
times.to_csv("times2021.csv", index = False)

# 3. Vitals

In [24]:
# define directories
vitals_directories = [ "/labs/kamaleswaranlab/MODS/Data/Emory_Data/em_data/2021/CJSEPSIS_VITALS_2021.dsv"]
# define thresholds
vitals_thresh = { "pulse": (0,250),
                 "spo2": (0,100),
                 "temperature": (25,45),
                 "sbp_cuff": (0,260),
                 "sbp_line": (0,260),
                 "dbp_cuff": (0, 220),
                 "dbp_line": (0, 220),
                 "map_cuff": (0,260),
                 "map_line": (0,260),
                 'unassisted_resp_rate': (0,80),
                 'end_tidal_co2': (0, 60),
                "o2_flow_rate": (0, 1000000),
                "height_cm": (0, 230),
                "daily_weight_kg": (0,300)}

# define columns to extract: always include ['pat_id', 'csn', 'recorded_time']
fetch_cols = ['pat_id', 'csn', 'recorded_time', 'o2_device'] + list(vitals_thresh.keys())

#import
vitals = import_dsv(directories = vitals_directories, fetch_cols = fetch_cols, thresholds = vitals_thresh, csns = csns, pat_ids = pat_ids)


Importing: 74:   1%|▏         | 74/5881 [04:28<5:51:24,  3.63s/it]
extraction complete


### 3.1 Vitals preprocessing

In [25]:
# replace unknown line values with cuff (giving higher priority to line)
vitals["sbp_line"] = vitals["sbp_line"].fillna(vitals["sbp_cuff"])
vitals["map_line"] = vitals["map_line"].fillna(vitals["map_cuff"])
vitals.drop(["sbp_cuff", "dbp_cuff", "map_cuff"], axis = 1, inplace = True)

# replace naming
vitals_rename = { "pulse": "HR",
                 "spo2": "O2Sat",
                 "temperature": "Temp",
                 "sbp_line": "SBP",
                 "map_line": "MAP",
                 "dbp_line": "DBP",
                 'unassisted_resp_rate': "Resp",
                 'end_tidal_co2': "EtCO2"}
vitals = vitals.rename(columns = vitals_rename)
# convert datatype
vitals["recorded_time"] = pd.to_datetime(vitals["recorded_time"], format = "%m/%d/%Y %H:%M:%S")


In [37]:
room_air = ['Room air', 
            'O2 not needed, Room air', 
            'Room air, Other: patient not wearing O2; repeatedly places O2 on forehead',
            'O2 standby, equipment at bedside, Room air',
            'O2 standby, equipment at bedside',
            'O2 standby, equipment at bedside, O2 not needed, Room air',
            'Room air, Other: walking',
            'Room air, Other: ambulated ed hallways',
            'Room air, Other: ambulatory',
            'Room air, Other: walking', 
            'Room air, Other:', 
            'Room air, Other: ambulation pulse ox',
            'Room air, Other: while ambulating on RA', 
            'Room air, Other: ambulatory sat', 
            'Room air, Other: AMBULATORY', 
            'Room air, Other: during ambulation',
            'Room air, Other: ambulatory saturation',
            'Room air, Other: on exertion',
            'Room air, Other: ambulatory pulse ox', 
            'Room air, Other: walking saturation of 91-93%', 
            'Room air, Other: ambulatory O2 sat',
            'Room air, Other: with ambulation',
            'Room air, Other: while ambulatory',
            'Room air, Other: when ambulating',
            'Room air, Other: pt smokes',
            'Room air, Other: with pt moving from wheelchair to bed',
            'Room air, Other: walk test',
            'Date\\Time Correction']
            
vitals.loc[~vitals.o2_device.isin(room_air), "o2_supp"] = 1
vitals.loc[vitals.o2_device.isna(), "o2_supp"] = np.nan
vitals["o2_supp"] = vitals.o2_supp.fillna(0)

In [38]:
vitals.to_csv("vitals_2021.csv", index = False)

## 4. Labs

In [27]:
# define directories
labs_directories = ["/labs/kamaleswaranlab/MODS/Data/Emory_Data/em_data/2021/CJSEPSIS_LABS_2021.dsv"]
# define thresholds
labs_thresh = { "pH": (6.7, 8),
              "PaCO2": (15, 150),
              "SaO2": (0,100),
              "AST": (0, 10000),
              "BUN": (0,200),
              "Alkalinephos": (0, 10000),
              "Calcium": (0,20),
              "Chloride": (60,150),
              "Creatinine": (0, 15),
              "Bilirubin_direct": (0,30),
              "Glucose": (0, 1200),
              "Lactate": (0,20),
              "Magnesium": (0,10), 
              "Phosphate": (0,20),
              "Potassium": (0,10),
              "Bilirubin_total": (0,30),
              "TroponinI": (0,15),
              "Hct": (0, 75),
               "Hgb": (0,25),
               "PTT": (0,150),
               "WBC": (0,150),
               "Fibrinogen": (0,1000),
               "Platelets": (0,1000),
               "Albumin": (0,20),
               "Anion_Gap": (0,100),
               "INR": (0,100),
               "MCHC": (0,100),
               "MCH": (0, 100),
               "MPV": (0, 100),
               "Phosphorus": (0, 100),
               "PT": (0, 300),
               "Protein": (0,20),
               "RBC": (0,20),
               "RDW-CV": (0,200),
               "RDW-SD": (0, 200)
              }

# import data
labs = import_labs(directories = labs_directories, thresholds = labs_thresh, csns = csns, 
                   pat_ids = pat_ids, loinc_dir = "../real_time_sepsis_development/real_time_data/emory_lab_loinc (3).csv")

# preprocess
# use uniform column name for times
labs = labs.rename(columns = {"collection_time": "recorded_time"})
# change recorded time datatype
labs["recorded_time"] = pd.to_datetime(labs["recorded_time"], format = "%m/%d/%Y %H:%M:%S")

Importing: 171:   1%|          | 171/27817 [07:07<19:12:52,  2.50s/it]
extraction complete


In [28]:
labs.to_csv("labs_2021.csv", index = False)

## 5. GCS

In [29]:
# specify directory
gcs_dir = ["/labs/kamaleswaranlab/MODS/Data/Emory_Data/em_data/2021/CJSEPSIS_GCS_2021.dsv"]

# specify columns: always include ['pat_id', 'csn', 'recorded_time']
fetch_cols = ["pat_id", "csn", "recorded_time", "gcs_total_score"]

gcs = import_dsv(directories = gcs_dir, fetch_cols = fetch_cols, pat_ids = pat_ids, csns = csns)

# preprocess
#gcs["recorded_time"] = pd.to_datetime(gcs["recorded_time"])
gcs["recorded_time"] = gcs["recorded_time"].apply(lambda x: datetime.strptime(x,"%m/%d/%Y %H:%M:%S"))

Importing: 11:   2%|▏         | 11/644 [00:03<03:21,  3.14it/s]
extraction complete


In [30]:
gcs.to_csv("GCS2021.csv", index = False)

## 6. Demographics

In [32]:
# specify directory
demo_dir = ["/labs/kamaleswaranlab/MODS/Data/Emory_Data/em_data/2018/CJSEPSIS_DEMOGRAPHICS_2018.dsv",
            "/labs/kamaleswaranlab/MODS/Data/Emory_Data/em_data/2019/CJSEPSIS_DEMOGRAPHICS_2019.dsv",
    "/labs/kamaleswaranlab/MODS/Data/Emory_Data/em_data/2020/CJSEPSIS_DEMOGRAPHICS_2020.dsv",
            "/labs/kamaleswaranlab/MODS/Data/Emory_Data/em_data/2021/CJSEPSIS_DEMOGRAPHICS_2021.dsv"]

# specify columns: always include pat_id
fetch_cols = ["pat_id", "race", "gender", "ethnicity"]

demographics = import_dsv(directories = demo_dir, fetch_cols = fetch_cols, pat_ids = pat_ids)


Importing: 3:   1%|          | 3/279 [00:01<02:15,  2.04it/s]
extraction complete


### 6.1 Demographics Preprocessing

In [33]:
demographics["race"] = demographics["race"].str.strip()
demographics["gender"] = demographics["gender"].str.strip()
dem = demographics.drop_duplicates(subset = ["pat_id", "race", "gender"])

race_code = {'Unknown, Unavailable or Unreported': 0,
            'African American  or Black': 3,
            'Caucasian or White':2, 
            'Native Hawaiian or Other Pacific Islander':0,
            'American Indian or Alaskan Native':0,
            'Multiple': 0, 
            'Asian': 1,
            'Patient Declines': 0, 
             'Not Recorded': 0, 
             'Alaskan Native': 0}

gender_code = {"Female": 1, "Male": 0}
dem = dem.replace({"gender": gender_code})
dem = dem.replace({"race": race_code})

race_gender = dem.groupby('pat_id').sum().reset_index()

race_gender = race_gender.rename(columns = {"gender": "is_female"})

race_gender.loc[race_gender.race == 0, "is_other"] =1
race_gender.loc[race_gender.race == 1, "is_asian"] =1
race_gender.loc[race_gender.race == 2, "is_white"] =1
race_gender.loc[race_gender.race == 3, "is_black"] =1

race_gender.drop(["race"], axis =1, inplace = True)
race_gender = race_gender.fillna(0)

In [35]:
race_gender.to_csv("race_gender_2021.csv", index= False)

## 7. Fluids
fluids data is stored in csv not dsv

In [36]:
fluids = import_csv(directories = ["/labs/kamaleswaranlab/MODS/CJSEPSIS_INOUTS_ALL.csv"], csns = csns)

Importing: 1415:   1%|          | 1415/135463 [27:58<44:10:02,  1.19s/it] 
extraction complete


### 7.1 Urine Output

In [39]:
urine = ['Urine Voided mL','Urine Straight Catheter Ouput']
urine_output = fluids[fluids.STRUCTURED_RESULT_TYPE.isin(urine)]

# column name change
urine_output = urine_output.rename(columns = {"CSN": "csn", "SERVICE_TIMESTAMP": "recorded_time", "RESULT_VAL": "urine_output"})
# data type change
urine_output["recorded_time"] = pd.to_datetime(urine_output["recorded_time"], format = "%m/%d/%Y %H:%M:%S")

urine_output= urine_output[["csn", "recorded_time", "urine_output"]]

urine_output.to_csv("urine_output2021.csv", index = False)

## 8. Mechanical Ventillation

In [40]:
# specify directory
vent_dir = ["/labs/kamaleswaranlab/MODS/Data/Emory_Data/em_data/2021/CJSEPSIS_VENT_2021.dsv"]

fetch_cols = ["pat_id", "csn", "recorded_time", "vent_start_time", "vent_stop_time", "vent_rate_set", "vent_tidal_rate_set", "vent_tidal_rate_exhaled" , "peep", "fio2"]

total_vent = import_dsv(directories = vent_dir, fetch_cols = fetch_cols, csns = csns)

total_vent = total_vent.rename(columns = {"fio2": "vent_fio2"})

total_vent["vent_start_time"] = pd.to_datetime(total_vent["vent_start_time"], format = "%m/%d/%Y %H:%M:%S")
total_vent["recorded_time"] = pd.to_datetime(total_vent["recorded_time"], format = "%m/%d/%Y %H:%M:%S")
total_vent["vent_stop_time"] = pd.to_datetime(total_vent["vent_stop_time"], format = "%m/%d/%Y %H:%M:%S")

Importing: 3:   1%|          | 3/312 [00:01<03:05,  1.66it/s]
extraction complete


### 8.1 MV Preprocessing

In [41]:
c_vent = total_vent[~total_vent["vent_start_time"].isna()].copy()

c_vent = c_vent.sort_values(by = "recorded_time")
v_start = c_vent.groupby(by = ["pat_id", "csn"]).first().reset_index()
v_start = v_start[["pat_id", "csn", "recorded_time"]].rename(columns = {"recorded_time": "vent_start_time"})
v_stop = c_vent.groupby(by = ["pat_id", "csn"]).last().reset_index()
v_stop = v_stop[["pat_id", "csn", "recorded_time"]].rename(columns = {"recorded_time": "vent_stop_time"})

new_vent = c_vent.drop(["vent_start_time", "vent_stop_time"], axis = 1)
new_vent = new_vent.merge(v_start, on = ["pat_id", "csn"], how = "left")
new_vent = new_vent.merge(v_stop, on = ["pat_id", "csn"], how = "left")

clean_vent = new_vent.copy()
clean_vent = clean_vent[clean_vent.csn.isin(csns)]
vent_values = ["vent_rate_set", "vent_tidal_rate_set", "vent_tidal_rate_exhaled", "peep", "vent_fio2"]
clean_vent[vent_values] = clean_vent[vent_values].replace(r'\>|\<|\%|\/|\s','',regex=True)

for i in vent_values:
    clean_vent[i] = pd.to_numeric(clean_vent[i], errors = "coerce")

clean_vent = clean_vent[~clean_vent[vent_values].isna().all(axis = 1)]


In [42]:
clean_vent.to_csv("vent2021.csv", index= False)

## 9. Merge

In [43]:
# will not print following columns due being PHI
hide = ["csn", "pat_id", "age", "is_Female", "is_asian", "is_white", "is_black", "is_other"]

### 9.0 Read all CSVs

In [58]:
times = pd.read_csv('times2021.csv')
time_cols = ["ed_presentation_time", "hospital_admission_date_time", "hospital_discharge_date_time"]
for i in time_cols:
    times[i] = pd.to_datetime(times[i])
    print("converted")

# few errors found with ed_pres
times = times.drop(["ed_presentation_time"], axis = 1)
times = times.drop_duplicates()

converted
converted
converted


In [62]:
vitals = pd.read_csv('vitals_2021.csv')
time_cols = ["recorded_time"]
for i in time_cols:
    vitals[i] = pd.to_datetime(vitals[i])
    print("converted")

vitals= vitals.drop(["o2_device"], axis = 1)

vitals.loc[:, ~vitals.columns.isin(hide)].head(5)

converted


,recorded_time,HR,O2Sat,Temp,SBP,DBP,MAP,Resp,EtCO2,o2_flow_rate,height_cm,daily_weight_kg,o2_supp
0,2021-03-08 16:11:00,84.0000,nan,36.8000,175.0000,nan,nan,21.0000,nan,nan,nan,nan,0.0000
1,2021-03-08 17:38:00,91.0000,99.0000,nan,146.0000,nan,nan,16.0000,nan,nan,nan,nan,0.0000
2,2021-03-08 19:52:00,84.0000,nan,36.7000,134.0000,nan,nan,18.0000,nan,nan,nan,nan,0.0000
3,2021-08-03 09:34:00,76.0000,nan,36.7000,134.0000,nan,nan,18.0000,nan,nan,nan,nan,0.0000
4,2021-11-29 16:02:00,87.0000,96.0000,36.8000,185.0000,nan,nan,18.0000,nan,nan,nan,nan,0.0000


In [63]:
labs = pd.read_csv('labs_2021.csv')
labs_cols = ["recorded_time"]
for i in time_cols:
    labs[i] = pd.to_datetime(labs[i])
    print("converted")

labs.loc[:, ~labs.columns.isin(hide)].head(5)

converted


,recorded_time,AST,Albumin,Alkalinephos,Anion_Gap,BUN,BaseExcess,Bilirubin_direct,Bilirubin_total,Calcium,Chloride,Creatinine,FiO2,Fibrinogen,Glucose,HCO3,Hct,Hgb,INR,Lactate,MCH,MCHC,MPV,Magnesium,PT,PTT,PaCO2,PaO2,Phosphate,Phosphorus,Platelets,Potassium,Protein,RBC,RDW-CV,RDW-SD,SaO2,Sodium,TroponinI,WBC,pH
0,2021-06-20 20:17:00,18.0000,4.1000,64.0000,nan,21.0000,nan,nan,0.4000,9.3000,107.0000,nan,nan,nan,nan,nan,40.1000,nan,nan,nan,nan,nan,9.1000,1.9000,nan,nan,nan,nan,nan,nan,206.0000,nan,7.1000,4.1300,12.7000,45.0000,nan,138.0000,nan,3.6000,nan
1,2021-04-15 15:14:00,nan,nan,nan,nan,nan,nan,nan,nan,7.3000,102.0000,0.4800,nan,nan,110.0000,nan,32.9000,10.8000,nan,nan,nan,32.8000,10.3000,nan,nan,nan,nan,nan,nan,nan,257.0000,nan,nan,3.6100,12.5000,nan,nan,139.0000,nan,9.2000,nan
2,2021-04-15 18:13:00,nan,nan,nan,10.0000,11.0000,nan,nan,nan,7.6000,nan,0.4700,nan,nan,119.0000,nan,33.0000,nan,nan,nan,30.0000,nan,nan,nan,nan,nan,nan,nan,nan,nan,273.0000,3.4000,nan,3.6300,12.6000,nan,nan,nan,nan,nan,nan
3,2021-04-16 00:37:00,nan,nan,47.0000,nan,10.0000,nan,nan,0.8000,8.0000,99.0000,0.5300,nan,nan,113.0000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,3.6000,5.4000,nan,nan,nan,nan,136.0000,nan,nan,nan
4,2021-05-22 18:48:00,nan,3.8000,94.0000,9.0000,10.0000,nan,nan,0.3000,nan,95.0000,nan,nan,nan,nan,nan,nan,nan,nan,nan,27.9000,nan,8.5000,nan,nan,nan,nan,nan,nan,nan,nan,4.5000,7.2000,4.0900,14.2000,45.4000,nan,130.0000,nan,5.4000,nan


In [64]:
urine_output = pd.read_csv("urine_output2021.csv")
urine_output["recorded_time"] = pd.to_datetime(urine_output["recorded_time"])

urine_output.loc[:, ~urine_output.columns.isin(hide)].head(5)

,recorded_time,urine_output
0,2021-01-16 11:00:00,500.0000
1,2021-01-16 16:00:00,525.0000
2,2021-01-25 10:00:00,300.0000
3,2021-01-28 19:00:00,125.0000
4,2021-02-11 15:00:00,600.0000


In [65]:
gcs = pd.read_csv("GCS2021.csv")
gcs["recorded_time"] = pd.to_datetime(gcs["recorded_time"])
gcs.loc[:, ~gcs.columns.isin(hide)].head(5)

,recorded_time,gcs_total_score
0,2021-03-08 16:33:00,15.0000
1,2021-10-01 20:50:00,15.0000
2,2021-10-03 19:24:00,15.0000
3,2021-12-04 18:07:00,15.0000
4,2021-12-04 23:46:00,15.0000


In [67]:
clean_vent = pd.read_csv("vent2021.csv")

time_cols = ["recorded_time", "vent_start_time","vent_stop_time"]

for i in time_cols:
    clean_vent[i] = pd.to_datetime(clean_vent[i])
    print("converted")
    
clean_vent.loc[:, ~clean_vent.columns.isin(hide)].head(5)

converted
converted
converted


,recorded_time,vent_rate_set,vent_tidal_rate_set,vent_tidal_rate_exhaled,peep,vent_fio2,vent_start_time,vent_stop_time
0,2021-01-01 02:14:00,20.0000,420.0000,370.0000,14.0000,1.0000,2021-01-01 02:14:00,2021-02-11 08:02:00
1,2021-01-01 02:38:00,20.0000,420.0000,nan,14.0000,1.0000,2021-01-01 02:14:00,2021-02-11 08:02:00
2,2021-01-01 06:14:00,nan,nan,nan,16.0000,1.0000,2021-01-01 02:14:00,2021-02-11 08:02:00
3,2021-01-01 08:00:00,20.0000,500.0000,409.0000,16.0000,0.8000,2021-01-01 02:14:00,2021-02-11 08:02:00
4,2021-01-01 08:52:00,20.0000,500.0000,431.0000,16.0000,0.6000,2021-01-01 02:14:00,2021-02-11 08:02:00


### 9.1 Merge static features

In [68]:
times = times.drop_duplicates()
stat = pd.merge(times, race_gender, on = "pat_id", how = "right")
stat["discharge_to"] = stat["discharge_to"] == "EXPIRED"
stat = stat.rename(columns = {"discharge_to": "In_hospital_death"})
stat = stat.drop_duplicates()
stat.to_csv("stat_2021.csv", index = False)

### Redefine CSNs

In [71]:
sepsis_2021 = stat[~stat.t_sepsis3.isna()].csn.unique()
nosepsis_2021 = stat[stat.t_sepsis3.isna()].csn.unique()
np.save("sepsis_2021_csn.npy", sepsis_2021)
np.save("nosepsis_2021_csn.npy", nosepsis_2021)

csns = list(stat.csn.unique())
print("number of sepsis patients from 2021: ", len(sepsis_2021))
print("number of non-sepsis patients from 2021: ",len(nosepsis_2021))
print("total number of encounters in 2021 cohort: ", len(csns))

number of sepsis patients from 2021:  1734
number of non-sepsis patients from 2021:  111353
total number of encounters in 2021 cohort:  113087


### 9.2 Merge longitudinal data

In [72]:
# merge vitals and labs on csn, pat_id, recorded_time w/ union

vitals = vitals[vitals.csn.isin(csns)]
labs =labs[labs.csn.isin(csns)]
urine_output = urine_output[urine_output.csn.isin(csns)]
gcs =gcs[gcs.csn.isin(csns)]
new_vent = clean_vent[clean_vent.csn.isin(csns)]
new_vent = new_vent[["csn", "pat_id", "vent_start_time", "vent_stop_time"]].drop_duplicates()

merged = pd.merge(vitals,labs,on = ["csn", "pat_id", "recorded_time"], how = "outer")

merged = pd.merge(merged,urine_output,on = ["csn", "recorded_time"], how = "outer")

merged = pd.merge(merged, gcs, on = ["csn", "pat_id", "recorded_time"], how = "outer")


In [73]:
merged.to_csv("longitudinal_2021.csv", index = False)

In [74]:
merged.loc[:, ~merged.columns.isin(hide)].head(5)

,recorded_time,HR,O2Sat,Temp,SBP,DBP,MAP,Resp,EtCO2,o2_flow_rate,height_cm,daily_weight_kg,o2_supp,AST,Albumin,Alkalinephos,Anion_Gap,BUN,BaseExcess,Bilirubin_direct,Bilirubin_total,Calcium,Chloride,Creatinine,FiO2,Fibrinogen,Glucose,HCO3,Hct,Hgb,INR,Lactate,MCH,MCHC,MPV,Magnesium,PT,PTT,PaCO2,PaO2,Phosphate,Phosphorus,Platelets,Potassium,Protein,RBC,RDW-CV,RDW-SD,SaO2,Sodium,TroponinI,WBC,pH,urine_output,gcs_total_score
0,2021-03-08 16:11:00,84.0000,nan,36.8000,175.0000,nan,nan,21.0000,nan,nan,nan,nan,0.0000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,2021-03-08 17:38:00,91.0000,99.0000,nan,146.0000,nan,nan,16.0000,nan,nan,nan,nan,0.0000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2021-03-08 19:52:00,84.0000,nan,36.7000,134.0000,nan,nan,18.0000,nan,nan,nan,nan,0.0000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,2021-08-03 09:34:00,76.0000,nan,36.7000,134.0000,nan,nan,18.0000,nan,nan,nan,nan,0.0000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,2021-11-29 16:02:00,87.0000,96.0000,36.8000,185.0000,nan,nan,18.0000,nan,nan,nan,nan,0.0000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
